In [63]:
import torch
import torchvision

In [64]:
!pip install wandb

In [65]:
import wandb
wandb.init(project="fashionMnist")

accuracy,▁▅▆▅▆▆▇▇▇▇▇▇▇▇▇▇████
epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁
accuracy,0.47801
epochs,20
loss,1.57186


In [76]:
def calc_acc(preds,labels):
  _,pred_max=torch.max(preds,1)
  acc=torch.sum(pred_max==labels.data,dtype=torch.float64)/len(preds)
  return acc

In [77]:
class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=torch.nn.Linear(784,256)
    self.fc2=torch.nn.Linear(256,64)
    self.fc3=torch.nn.Linear(64,32)
    self.fc4=torch.nn.Linear(32,16)
    self.fc5=torch.nn.Linear(16,10)
  def forward(self,x):
    x=x.reshape((x.shape[0],784))
    x=self.fc1(x)
    x=torch.relu(x)
    x=self.fc2(x)
    x=torch.relu(x)
    x=self.fc3(x)
    x=torch.relu(x)
    x=self.fc4(x)
    x=torch.relu(x)
    x=self.fc5(x)
    x=torch.relu(x)
    return x


In [78]:
device=torch.device("cuda")
model=MyModel()
model=model.to(device)
model.train(True)

MyModel(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=10, bias=True)
)

In [79]:
#hyper parameter
batch_size=64
epochs=20
lr=0.01

In [80]:
transform=torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize((0),(1)),
])

dataset=torchvision.datasets.FashionMNIST("./dataset",train=True,transform=transform,download=True)
train_data_loader=torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [81]:
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
loss_function=torch.nn.CrossEntropyLoss()


In [82]:
for epoch in range(epochs):
  train_loss=0.0
  train_acc=0.0
  for images,labels in train_data_loader:
    images=images.to(device)
    labels=labels.to(device)

    optimizer.zero_grad()
    pred=model(images)
    loss=loss_function(pred,labels)
    loss.backward()
    optimizer.step()
    train_loss+=loss
    train_acc+=calc_acc(pred,labels)
  
  total_loss=train_loss/len(train_data_loader)
  total_acc=train_acc/len(train_data_loader)
  print(f"epochs: {epoch} , loss: {total_loss}, acc: {total_acc}")
  wandb.log({'epochs': epoch+1,
        'loss': total_loss,
        'accuracy':total_acc
        })

epochs: 0 , loss: 1.0473616123199463, acc: 0.626915644989339
epochs: 1 , loss: 0.8642200827598572, acc: 0.6863839285714286
epochs: 2 , loss: 0.8327764868736267, acc: 0.697378065031983
epochs: 3 , loss: 0.8131236433982849, acc: 0.7018590085287847
epochs: 4 , loss: 0.807710587978363, acc: 0.7027085554371002
epochs: 5 , loss: 0.7989712953567505, acc: 0.7060900852878464
epochs: 6 , loss: 0.8011348843574524, acc: 0.7048074360341151
epochs: 7 , loss: 0.7919422388076782, acc: 0.7073061034115139
epochs: 8 , loss: 0.7825784683227539, acc: 0.7091384594882729
epochs: 9 , loss: 0.7856165170669556, acc: 0.7083388859275053
epochs: 10 , loss: 0.7799783945083618, acc: 0.7084055170575693
epochs: 11 , loss: 0.7738140821456909, acc: 0.7119536247334755
epochs: 12 , loss: 0.7673687934875488, acc: 0.7125699626865671
epochs: 13 , loss: 0.7728363871574402, acc: 0.710954157782516
epochs: 14 , loss: 0.7715227007865906, acc: 0.711054104477612
epochs: 15 , loss: 0.7727746963500977, acc: 0.7096715085287847
epochs:

In [83]:
torch.save(model.state_dict(),"fashion.pth")

In [84]:
#infrence
import cv2
import numpy as np

In [85]:
model.eval()
img=cv2.imread('test1.png')
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.resize(img,(28,28))
tensor=transform(img).unsqueeze(0)
tensor=tensor.to(device)
pred=model(tensor)
# pred=pred.argmax()

pred=pred.cpu().detach().numpy()
out=np.argmax(pred)
out

2